In [ ]:
from pathlib import Path
import sys
import os
import django
from asgiref.sync import sync_to_async
import pandas as pd

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()


@sync_to_async
def to_run():
    return


res = await to_run()

In [ ]:
res

In [3]:
import pandas as pd

d_1 = {"color": {"first": [1, 2], "second": None}, "date": "2021-01-10"}
d_2 = {"color": {"first": [1, 2], "second": None}, "date": "2021-01-10"}

d_1 == d_2

True